In [161]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna

In [162]:
input = pd.read_csv('some_data.csv')

In [163]:
input

,Id,running_time,route_distance_km,sequence,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,...,windgusts_10m (km/h),start_node_lat,start_node_lon,end_node_lat,end_node_lon,distance_haversine,distance_dummy_manhattan,direction,central_lon,central_lat
0,6198,2022-01-24 03:38:30,4.744,"[4773237812, 4773237811, 2059504508, 205950375...",0,0,0,1,0,0,...,39.2,46.429778,30.721539,46.459480,30.739424,3.575801,4.673509,22.526888,30.730481,46.444629
1,6417,2022-01-24 03:45:51,6.279,"[5020979245, 290008230, 703126946, 290008229, ...",0,0,0,1,0,0,...,39.2,46.477159,30.733953,46.440505,30.758535,4.489608,5.957995,155.194004,30.746244,46.458832
2,7054,2022-01-24 03:52:14,3.934,"[4363705737, 4362696212, 317184766, 4768027647...",0,0,0,1,0,0,...,39.2,46.484869,30.716282,46.460031,30.721009,2.785502,3.123779,172.531504,30.718645,46.472450
3,9628,2022-01-24 04:03:21,5.959,"[3292022393, 3292022394, 701988912, 27126496, ...",0,0,0,1,0,0,...,39.2,46.399419,30.723280,46.468524,30.738275,7.769541,8.833946,8.499896,30.730777,46.433972
4,10283,2022-01-24 04:01:12,7.028,"[1920586755, 1920586759, 2350039045, 392526370...",0,0,0,1,0,0,...,39.2,46.408082,30.705124,46.432172,30.760893,5.044822,6.954542,57.908292,30.733008,46.420127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,525706,2022-01-24 18:46:17,2.897,"[4775841528, 4775841529, 3127870582, 477584152...",0,1,0,0,0,1,...,40.7,46.429601,30.710972,46.415021,30.722073,1.830921,2.471968,152.302901,30.716522,46.422311
996,526604,2022-01-24 18:46:44,3.482,"[2041636927, 352914232, 1570776571, 1570776566...",0,1,0,0,0,1,...,40.7,46.440041,30.725005,46.441431,30.695256,2.284715,2.434133,-86.108523,30.710130,46.440736
997,527213,2022-01-24 18:47:25,3.486,"[6046083886, 6046063725, 6046063732, 604606373...",0,1,0,0,0,1,...,40.7,46.492118,30.726941,46.473745,30.744623,2.450796,3.396525,146.461626,30.735782,46.482931
998,527520,2022-01-24 18:52:01,0.703,"[10980434, 5117981736, 5966955815, 5966955819,...",0,1,0,0,0,1,...,40.7,46.471075,30.741119,46.469035,30.748650,0.619775,0.803652,111.475259,30.744884,46.470055


In [164]:
nodes_info = pd.read_csv('nodes_final_test_info_clustered.csv', index_col="node_id")

In [165]:
nodes_info.head()

,lat,lon,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,...,Cluster 20,Cluster 21,Cluster 22,Cluster 23,Cluster 24,Cluster 25,Cluster 26,Cluster 27,Cluster 28,Cluster 29
node_id,,,,,,,,,,,,,,,,,,,,,
10980417,46.468524,30.738275,0.000709,0.000482,0.000037,0.000625,0.000025,0.000008,0.000093,0.000011,...,0.000023,0.000011,0.000044,0.000028,0.000045,0.000449,0.000128,0.995967,0.000018,0.000043
10980418,46.469605,30.738516,0.007064,0.004538,0.000306,0.005469,0.000209,0.000069,0.000749,0.000096,...,0.000192,0.000092,0.000362,0.000229,0.000368,0.003436,0.001027,0.964441,0.000150,0.000355
10980419,46.469758,30.736733,0.024639,0.019119,0.001162,0.026448,0.000833,0.000255,0.002888,0.000359,...,0.000729,0.000359,0.001397,0.000881,0.001368,0.012075,0.003732,0.862288,0.000572,0.001426
10980421,46.471836,30.730899,0.069186,0.108914,0.004096,0.347663,0.003649,0.000907,0.010214,0.001309,...,0.002599,0.001468,0.005186,0.003264,0.004475,0.029538,0.010997,0.236537,0.002102,0.006339
10980422,46.473256,30.731101,0.075225,0.131249,0.003721,0.370207,0.003440,0.000874,0.009050,0.001271,...,0.002382,0.001378,0.004686,0.002979,0.004078,0.025642,0.009845,0.201598,0.001933,0.005851


In [118]:
sequences = pd.read_csv("orders_with_sequencesv2.csv", index_col="Unnamed: 0")

In [12]:
input['sequence'] = sequences['sequence']

In [166]:
def get_start_node(row):
    sequences = row['sequence']
    if sequences == "Zero normal points":
        return 0
    else:
        sequences = eval(sequences)
        if type(sequences) is list:
            return sequences[0]
        else:
            return sequences
        
def get_finish_node(row):
    sequences = row['sequence']
    if sequences == "Zero normal points":
        return 0
    else:
        sequences = eval(sequences)
        if type(sequences) is list:
            return sequences[-1]
        else:
            return sequences
        
def get_center_node(row):
    sequences = row['sequence']
    if sequences == "Zero normal points":
        return 0
    else:
        sequences = eval(sequences)
        if type(sequences) is list:
            return sequences[len(sequences)//2]
        else:
            return sequences

In [167]:
input['node_start'] = input.apply(get_start_node, axis = 1)
input['node_finish'] = input.apply(get_finish_node, axis = 1)
input['node_center'] = input.apply(get_center_node, axis = 1)

In [168]:
input

,Id,running_time,route_distance_km,sequence,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,...,end_node_lat,end_node_lon,distance_haversine,distance_dummy_manhattan,direction,central_lon,central_lat,node_start,node_finish,node_center
0,6198,2022-01-24 03:38:30,4.744,"[4773237812, 4773237811, 2059504508, 205950375...",0,0,0,1,0,0,...,46.459480,30.739424,3.575801,4.673509,22.526888,30.730481,46.444629,4773237812,290913575,5214583010
1,6417,2022-01-24 03:45:51,6.279,"[5020979245, 290008230, 703126946, 290008229, ...",0,0,0,1,0,0,...,46.440505,30.758535,4.489608,5.957995,155.194004,30.746244,46.458832,5020979245,2408742772,1988203318
2,7054,2022-01-24 03:52:14,3.934,"[4363705737, 4362696212, 317184766, 4768027647...",0,0,0,1,0,0,...,46.460031,30.721009,2.785502,3.123779,172.531504,30.718645,46.472450,4363705737,3127846438,317473325
3,9628,2022-01-24 04:03:21,5.959,"[3292022393, 3292022394, 701988912, 27126496, ...",0,0,0,1,0,0,...,46.468524,30.738275,7.769541,8.833946,8.499896,30.730777,46.433972,3292022393,8952478905,1567415713
4,10283,2022-01-24 04:01:12,7.028,"[1920586755, 1920586759, 2350039045, 392526370...",0,0,0,1,0,0,...,46.432172,30.760893,5.044822,6.954542,57.908292,30.733008,46.420127,1920586755,27126488,1987538297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,525706,2022-01-24 18:46:17,2.897,"[4775841528, 4775841529, 3127870582, 477584152...",0,1,0,0,0,1,...,46.415021,30.722073,1.830921,2.471968,152.302901,30.716522,46.422311,4775841528,2034419510,290800924
996,526604,2022-01-24 18:46:44,3.482,"[2041636927, 352914232, 1570776571, 1570776566...",0,1,0,0,0,1,...,46.441431,30.695256,2.284715,2.434133,-86.108523,30.710130,46.440736,2041636927,4775833861,4775833772
997,527213,2022-01-24 18:47:25,3.486,"[6046083886, 6046063725, 6046063732, 604606373...",0,1,0,0,0,1,...,46.473745,30.744623,2.450796,3.396525,146.461626,30.735782,46.482931,6046083886,5206802325,4807102911
998,527520,2022-01-24 18:52:01,0.703,"[10980434, 5117981736, 5966955815, 5966955819,...",0,1,0,0,0,1,...,46.469035,30.748650,0.619775,0.803652,111.475259,30.744884,46.470055,10980434,1570776901,4491475141


In [169]:
input_start_finish = input.join(nodes_info.drop(['lat', 'lon'], axis=1), on='node_start')
input_start_finish = input_start_finish.rename({f"Cluster {i}" : f"start_cluster_{i}" for i in range(30)}, axis=1)
input_start_finish = input_start_finish.join(nodes_info.drop(['lat', 'lon'], axis=1), on='node_finish')
input_start_finish = input_start_finish.rename({f"Cluster {i}" : f"finish_cluster_{i}" for i in range(30)}, axis=1)
input_start_finish

,Id,running_time,route_distance_km,sequence,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,...,finish_cluster_20,finish_cluster_21,finish_cluster_22,finish_cluster_23,finish_cluster_24,finish_cluster_25,finish_cluster_26,finish_cluster_27,finish_cluster_28,finish_cluster_29
0,6198,2022-01-24 03:38:30,4.744,"[4773237812, 4773237811, 2059504508, 205950375...",0,0,0,1,0,0,...,0.004806,0.001797,0.009580,0.005707,0.010375,0.280568,0.038456,0.292387,0.003571,0.007107
1,6417,2022-01-24 03:45:51,6.279,"[5020979245, 290008230, 703126946, 290008229, ...",0,0,0,1,0,0,...,0.006799,0.001337,0.009413,0.006234,0.030729,0.069401,0.219735,0.020081,0.004238,0.003938
2,7054,2022-01-24 03:52:14,3.934,"[4363705737, 4362696212, 317184766, 4768027647...",0,0,0,1,0,0,...,0.007255,0.003842,0.018298,0.010219,0.011255,0.058645,0.026293,0.091015,0.005962,0.023096
3,9628,2022-01-24 04:03:21,5.959,"[3292022393, 3292022394, 701988912, 27126496, ...",0,0,0,1,0,0,...,0.000023,0.000011,0.000044,0.000028,0.000045,0.000449,0.000128,0.995967,0.000018,0.000043
4,10283,2022-01-24 04:01:12,7.028,"[1920586755, 1920586759, 2350039045, 392526370...",0,0,0,1,0,0,...,0.001017,0.000158,0.001219,0.000844,0.006827,0.004387,0.017881,0.001615,0.000591,0.000439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,525706,2022-01-24 18:46:17,2.897,"[4775841528, 4775841529, 3127870582, 477584152...",0,1,0,0,0,1,...,0.033766,0.002180,0.445265,0.073512,0.023582,0.012327,0.018287,0.007104,0.019420,0.007400
996,526604,2022-01-24 18:46:44,3.482,"[2041636927, 352914232, 1570776571, 1570776566...",0,1,0,0,0,1,...,0.009910,0.007725,0.035252,0.020082,0.008938,0.014616,0.012271,0.013832,0.010164,0.129869
997,527213,2022-01-24 18:47:25,3.486,"[6046083886, 6046063725, 6046063732, 604606373...",0,1,0,0,0,1,...,0.002565,0.001238,0.004437,0.002940,0.005049,0.037410,0.013366,0.288832,0.001992,0.004267
998,527520,2022-01-24 18:52:01,0.703,"[10980434, 5117981736, 5966955815, 5966955819,...",0,1,0,0,0,1,...,0.002940,0.001239,0.004964,0.003274,0.006314,0.058501,0.018680,0.188601,0.002218,0.004145


In [170]:
input_start_finish_center = input_start_finish.join(nodes_info.drop(['lat', 'lon'], axis=1), on='node_center')
input_start_finish_center = input_start_finish_center.rename({f"Cluster {i}" : f"center_cluster_{i}" for i in range(30)}, axis=1)
input_start_finish_center

,Id,running_time,route_distance_km,sequence,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,...,center_cluster_20,center_cluster_21,center_cluster_22,center_cluster_23,center_cluster_24,center_cluster_25,center_cluster_26,center_cluster_27,center_cluster_28,center_cluster_29
0,6198,2022-01-24 03:38:30,4.744,"[4773237812, 4773237811, 2059504508, 205950375...",0,0,0,1,0,0,...,0.013717,0.003685,0.045085,0.019603,0.024870,0.101205,0.069110,0.046614,0.009976,0.018058
1,6417,2022-01-24 03:45:51,6.279,"[5020979245, 290008230, 703126946, 290008229, ...",0,0,0,1,0,0,...,0.004839,0.001510,0.010368,0.005796,0.011196,0.590644,0.052000,0.075470,0.003472,0.006112
2,7054,2022-01-24 03:52:14,3.934,"[4363705737, 4362696212, 317184766, 4768027647...",0,0,0,1,0,0,...,0.002377,0.001551,0.005112,0.003165,0.003691,0.018325,0.008171,0.056137,0.001991,0.008026
3,9628,2022-01-24 04:03:21,5.959,"[3292022393, 3292022394, 701988912, 27126496, ...",0,0,0,1,0,0,...,0.029077,0.001871,0.546919,0.073559,0.018087,0.009697,0.014032,0.005721,0.017462,0.006324
4,10283,2022-01-24 04:01:12,7.028,"[1920586755, 1920586759, 2350039045, 392526370...",0,0,0,1,0,0,...,0.034722,0.002592,0.326604,0.063083,0.032142,0.017629,0.026947,0.009693,0.019732,0.009098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,525706,2022-01-24 18:46:17,2.897,"[4775841528, 4775841529, 3127870582, 477584152...",0,1,0,0,0,1,...,0.025970,0.006627,0.256855,0.080690,0.018308,0.019381,0.020553,0.014207,0.025768,0.033631
996,526604,2022-01-24 18:46:44,3.482,"[2041636927, 352914232, 1570776571, 1570776566...",0,1,0,0,0,1,...,0.008630,0.003294,0.041832,0.016570,0.009588,0.018551,0.015418,0.014385,0.007489,0.024628
997,527213,2022-01-24 18:47:25,3.486,"[6046083886, 6046063725, 6046063732, 604606373...",0,1,0,0,0,1,...,0.001415,0.000915,0.002636,0.001745,0.002349,0.011846,0.005221,0.062854,0.001166,0.003596
998,527520,2022-01-24 18:52:01,0.703,"[10980434, 5117981736, 5966955815, 5966955819,...",0,1,0,0,0,1,...,0.002519,0.001153,0.004433,0.002893,0.005089,0.045375,0.014340,0.450485,0.001938,0.004059


In [159]:
# input_start_finish = input_start_finish.drop(['node_start', 'node_finish', 'node_center', 'sequence'], axis=1)
# input_start_finish_center = input_start_finish_center.drop(['node_start', 'node_finish', 'node_center', 'sequence'], axis=1)

In [171]:
# input_start_finish.to_csv('test_with_clustering_start_finish.csv', index=False)
input_start_finish_center.to_csv('Really_final_test_with_clustering_start_center_finish.csv', index=False)

In [121]:
input_start_finish_center = pd.read_csv('final_df_start_center_finish_pca.csv', index_col="Unnamed: 0")

In [122]:
input_start_finish_center = input_start_finish_center.set_index('Id')

In [123]:
input_start_finish_center

,running_time,completed_time,route_distance_km,delta_time,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,...,center_cluster_26,center_cluster_27,center_cluster_28,center_cluster_29,start_pca0,start_pca1,finish_pca0,finish_pca1,center_pca0,center_pca1
Id,,,,,,,,,,,,,,,,,,,,,
-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,0,1,0,0,0,1,...,0.001009,0.965341,0.000146,0.000343,0.022674,0.002687,-0.003298,-0.006111,0.009688,-0.001712
-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0,0,0,1,0,0,0,...,0.002630,0.028300,0.000603,0.002082,0.020041,-0.007313,-0.004006,0.015119,0.008018,0.003903
-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0,0,0,1,0,0,0,...,0.009700,0.232100,0.001619,0.003763,-0.002792,-0.006090,0.031984,-0.002009,0.014596,-0.004049
5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0,1,0,0,0,0,0,...,0.001445,0.002135,0.000686,0.001092,0.133338,-0.016324,0.123455,0.004813,0.128397,-0.005755
1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0,0,0,1,0,0,0,...,0.001769,0.002392,0.001492,0.896434,-0.024061,0.013013,-0.041836,0.037430,-0.032948,0.025221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7096714159023973792,2022-01-24 21:10:38,2022-01-24 21:23:53,7.397,795.0,0,0,0,1,0,0,...,0.035864,0.103321,0.003310,0.005211,0.010001,-0.005469,-0.039235,0.014532,-0.014617,0.004531
-3836026425968071806,2022-01-24 15:10:27,2022-01-24 15:20:21,1.948,594.0,1,0,0,0,0,0,...,0.214723,0.037090,0.004636,0.005904,-0.018507,-0.040103,-0.018507,-0.040103,-0.018507,-0.040103
2926216435675216636,2022-01-24 13:57:04,2022-01-24 14:03:18,2.547,374.0,1,0,0,0,0,0,...,0.020623,0.014552,0.026120,0.036224,0.008734,0.017353,-0.000601,0.040633,0.004066,0.028993


In [126]:
pred_time = pd.read_csv('pred_time.csv', index_col="Id")
pred_time.head()

,pred_time
Id,
-4773019581999572651,445.785429
-7575630690398473489,339.653051
-6264582368520213833,555.164520
5964315354301636538,327.574647
1372379574816145639,410.407676


In [127]:
input_start_finish_center["pred_time"] = pred_time

In [128]:
input_start_finish_center

,running_time,completed_time,route_distance_km,delta_time,is_afternoon,is_evening,is_morning,is_night,morning_peak_time,evening_peak_time,...,center_cluster_27,center_cluster_28,center_cluster_29,start_pca0,start_pca1,finish_pca0,finish_pca1,center_pca0,center_pca1,pred_time
Id,,,,,,,,,,,,,,,,,,,,,
-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,0,1,0,0,0,1,...,0.965341,0.000146,0.000343,0.022674,0.002687,-0.003298,-0.006111,0.009688,-0.001712,445.785429
-7575630690398473489,2022-01-24 06:53:53,2022-01-24 07:06:26,3.526,753.0,0,0,1,0,0,0,...,0.028300,0.000603,0.002082,0.020041,-0.007313,-0.004006,0.015119,0.008018,0.003903,339.653051
-6264582368520213833,2022-01-24 10:00:59,2022-01-24 10:15:58,5.071,899.0,0,0,1,0,0,0,...,0.232100,0.001619,0.003763,-0.002792,-0.006090,0.031984,-0.002009,0.014596,-0.004049,555.164520
5964315354301636538,2022-01-24 14:28:05,2022-01-24 14:35:08,2.867,423.0,1,0,0,0,0,0,...,0.002135,0.000686,0.001092,0.133338,-0.016324,0.123455,0.004813,0.128397,-0.005755,327.574647
1372379574816145639,2022-01-24 11:57:29,2022-01-24 12:06:29,3.751,540.0,0,0,1,0,0,0,...,0.002392,0.001492,0.896434,-0.024061,0.013013,-0.041836,0.037430,-0.032948,0.025221,410.407676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7096714159023973792,2022-01-24 21:10:38,2022-01-24 21:23:53,7.397,795.0,0,0,0,1,0,0,...,0.103321,0.003310,0.005211,0.010001,-0.005469,-0.039235,0.014532,-0.014617,0.004531,548.510212
-3836026425968071806,2022-01-24 15:10:27,2022-01-24 15:20:21,1.948,594.0,1,0,0,0,0,0,...,0.037090,0.004636,0.005904,-0.018507,-0.040103,-0.018507,-0.040103,-0.018507,-0.040103,342.160014
2926216435675216636,2022-01-24 13:57:04,2022-01-24 14:03:18,2.547,374.0,1,0,0,0,0,0,...,0.014552,0.026120,0.036224,0.008734,0.017353,-0.000601,0.040633,0.004066,0.028993,267.563772


In [129]:
input_start_finish_center = input_start_finish_center.drop(['running_time', 'completed_time'], axis=1)

In [130]:
y_train = input_start_finish_center['delta_time']
X_train = input_start_finish_center.drop(['delta_time'], axis=1)

In [131]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
optuna.logging.set_verbosity(optuna.logging.ERROR)
from lightgbm import early_stopping

In [132]:
def objective(trial):
    
    param = {
        'metric': 'rmse', 
        'random_state': 48,
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
        X_train_1, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_1, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]
        
        model = lgb.LGBMRegressor(**param)  
            
        model.fit(X_train_1,y_train_1,eval_set=[(X_val,y_val)], callbacks=[early_stopping(stopping_rounds=100, verbose=False)])
        
        y_pred = model.predict(X_val)

        cv_scores[idx] = mean_squared_error(y_val, y_pred, squared=False)

    return np.mean(cv_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, show_progress_bar=True)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/20 [00:00<?, ?it/s]

Number of finished trials: 20
Best trial: {'reg_alpha': 0.22608828217534827, 'reg_lambda': 0.0011941851969042886, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.02, 'max_depth': 20, 'num_leaves': 203, 'min_child_samples': 84, 'min_data_per_groups': 79}


In [133]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

Number of finished trials: 20
Best trial: {'reg_alpha': 0.22608828217534827, 'reg_lambda': 0.0011941851969042886, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.02, 'max_depth': 20, 'num_leaves': 203, 'min_child_samples': 84, 'min_data_per_groups': 79}


In [134]:
study.best_trial

FrozenTrial(number=16, values=[118.64950205991306], datetime_start=datetime.datetime(2023, 3, 5, 7, 15, 41, 831486), datetime_complete=datetime.datetime(2023, 3, 5, 7, 15, 58, 812131), params={'reg_alpha': 0.22608828217534827, 'reg_lambda': 0.0011941851969042886, 'colsample_bytree': 0.9, 'subsample': 0.5, 'learning_rate': 0.02, 'max_depth': 20, 'num_leaves': 203, 'min_child_samples': 84, 'min_data_per_groups': 79}, distributions={'reg_alpha': LogUniformDistribution(high=10.0, low=0.001), 'reg_lambda': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': CategoricalDistribution(choices=(0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)), 'subsample': CategoricalDistribution(choices=(0.4, 0.5, 0.6, 0.7, 0.8, 1.0)), 'learning_rate': CategoricalDistribution(choices=(0.006, 0.008, 0.01, 0.014, 0.017, 0.02)), 'max_depth': CategoricalDistribution(choices=(10, 20, 100)), 'num_leaves': IntUniformDistribution(high=1000, low=1, step=1), 'min_child_samples': IntUniformDistribution(high=300, low